# Optuna Hyperparameter Optimisation

This notebook is used to perform the Hyperparameter Optimisation (HO) of the centralised model on centralised data. The centralised model serves as a reference of the gold standard that can be achieved in Federated Learning. For this reason, we also run HO in the centralised data. The code of this notebook uses an interface that we developed for communication with [Optuna](https://optuna.org/).

## Imports

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..' + os.sep + '..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
import optuna
from experiment_parameters.TrainerFactory import dataset_model_dictionary
import xgboost as xgb

/home/jose/miniconda3/envs/flower_py3-10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-14 14:10:38.530929: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-14 14:10:38.537548: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-14 14:10:38.547046: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-1

In [3]:
from experiment_parameters.model_builder.ModelBuilder import Director, get_training_configuration
from metrics.Metrics import DictOfMetrics
from metrics.Evaluator import evaluator

## Model Factory

We use the director pattern for generating models with different configurations given by Optuna. The hyperparameters of the model are given by optuna using _trial_. We use [Cross Entropy Loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html#sklearn.metrics.log_loss) for classification and [Median Absolute Error](https://en.wikipedia.org/wiki/Mean_absolute_error) for regression.

In [4]:
director = Director()

In [5]:
def get_parameters(trial, model_type):
    parameters = get_training_configuration(trial=trial, model_type=model_type)
    return parameters

In [6]:
def get_mlp(input_dim, num_classes, parameters):
    return director.create_mlp(input_parameters=input_dim, num_classes=num_classes, parameters=parameters)

In [7]:
def get_xgboost_tree(input_dim, num_classes, parameters):
    return director.create_xgboost(input_parameters=input_dim, num_classes=num_classes, parameters=parameters)

## MLP Wine

We offer a quick training using the Wine dataset, so you can check if the code works.

In [8]:
metric_list = ["CrossEntropyLoss"]

In [9]:
dataset_factory = dataset_model_dictionary["wine"]()
X_train, y_train = dataset_factory.get_dataset().get_training_data()
X_test, y_test = dataset_factory.get_dataset().get_test_data()

In [10]:
def wine_mlp_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="mlp")
    ml_model = get_mlp(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    ml_model.fit(X_train, y_train, batch_size=parameters["batch_size"]) 
    metrics: DictOfMetrics = evaluator(X_train, y_train, ml_model, metric_list)
    return metrics.get_value_of_metric("CrossEntropyLoss")

In [11]:
from util.OptunaConnection import optuna_create_study

# 3. Create a study object and optimize the objective function.
study = optuna_create_study("mlp_wine", direction=['minimize'])
study.optimize(wine_mlp_optimization, n_trials=60)

[I 2026-01-14 14:10:41,039] A new study created in RDB with name: mlp_wine
2026-01-14 14:10:43.669319: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-01-14 14:10:43.697001: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-01-14 14:10:43.698110: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github

InternalError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE' [Op:Mul] name: 

## MLP HAR

This is the code for optimisation on the Human Activity Recognition(HAR) dataset.

In [ ]:
dataset_factory = dataset_model_dictionary["har"]()
X_train, y_train = dataset_factory.get_dataset().get_training_data()
X_test, y_test = dataset_factory.get_dataset().get_test_data()

In [ ]:
def har_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="mlp")
    ml_model = get_mlp(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    ml_model.fit(X_train, y_train, epochs=20, batch_size=parameters["batch_size"]) 
    metrics: DictOfMetrics = evaluator(X_train, y_train, ml_model, metric_list)
    return metrics.get_value_of_metric("CrossEntropyLoss")

In [ ]:
from util.OptunaConnection import optuna_create_study

# 3. Create a study object and optimize the objective function.
study = optuna_create_study("mlp_har", direction=['minimize'])
study.optimize(har_optimization, n_trials=60)

## MLP Edge-IIOT-Coreset

This is the code for optimisation on the IIoT attack dataset, with the reduction using the _Coreset_ method.

In [ ]:
## Grid Search MLP HAR
from experiment_parameters.TrainerFactory import dataset_model_dictionary
from util.OptunaConnection import optuna_create_study

dataset_factory = dataset_model_dictionary["edge-iot-coreset"]()
X_train, y_train = dataset_factory.get_dataset().get_training_data()
X_test, y_test = dataset_factory.get_dataset().get_test_data()

In [ ]:
def edge_iiot_coreset_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="mlp")
    ml_model = get_mlp(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    ml_model.fit(X_train, y_train, epochs=30, batch_size=parameters["batch_size"]) 
    metrics: DictOfMetrics = evaluator(X_train, y_train, ml_model, metric_list)
    return metrics.get_value_of_metric("CrossEntropyLoss")

In [ ]:
study = optuna_create_study("mlp_edge_iiot_coreset", direction=['minimize'])
study.optimize(edge_iiot_coreset_optimization, n_trials=30)

## MLP Electric Consumption

This is the code for optimisation on the Electric Consumption dataset.

In [ ]:
## Grid Search MLP HAR
from experiment_parameters.TrainerFactory import dataset_model_dictionary
from util.OptunaConnection import optuna_create_study

metric_list = ["MAE"]

dataset_factory = dataset_model_dictionary["electric-consumption"]()
X_train, y_train = dataset_factory.get_dataset().get_training_data()
X_test, y_test = dataset_factory.get_dataset().get_test_data()

In [ ]:
def electric_consumption_optimisation(trial):
    parameters = get_parameters(trial=trial, model_type="mlp")
    shape: int
    try: shape = y_train.shape[1] 
    except: shape = 1
    ml_model = get_mlp(input_dim=X_train.shape[1], num_classes=shape, parameters=parameters)
    ml_model.fit(X_train, y_train, batch_size=parameters["batch_size"]) 
    metrics: DictOfMetrics = evaluator(X_train, y_train, ml_model, metric_list)
    return metrics.get_value_of_metric("MAE")

In [ ]:
study = optuna_create_study("mlp_electric_consumption", direction=['minimize'])
study.optimize(electric_consumption_optimisation, n_trials=30)